# Gráfico 1: Cruzamentos mais perigosos

### Coletando as geometrias das ruas de Porto Velho

In [14]:
from osmnx import geometries_from_address
import geopandas as gpd
import warnings

In [15]:
# Carrega dados de acidentes
gdf = gpd.read_file('Acidentes_PortoVelho.geojson')

In [16]:
# tags: https://wiki.openstreetmap.org/wiki/Map_features
osm_pvh = geometries_from_address("Porto Velho, Rondonia, Brazil",
                                  tags={'place': 'city',
                                        'highway': ['trunk', 'secondary', 'residential',
                                                    'tertiary', 'service', 'primary',
                                                    'primary_link', 'trunk_link', 'unclassified',
                                                    'track', 'path', 'living_street',
                                                    'footway', 'tertiary_link', 'secondary_link',
                                                    'cycleway', 'services', 'proposed', 'pedestrian']}
                                  , dist=20000)

pvh = osm_pvh[['name', 'geometry', 'highway', 'alt_name', 'oneway', 'bridge', 'layer', 'surface']]
pvh = pvh.reset_index()
# Mudar crs
pvh = pvh.to_crs(epsg=4674)

# Pegar localização da cidade de Porto Velho
ponto_pvh = pvh[pvh['name'] == 'Porto Velho']
# Deleta multiindex node e numero do osm
ponto_pvh = ponto_pvh.reset_index()
# Cria GDF para o buffer (apenas para comodidade)
buffer_area_interesse = gpd.GeoDataFrame(ponto_pvh['name'],
                                         geometry=ponto_pvh.buffer(0.10),
                                         crs=4674)
# Deleta ponto e deleta a sobra de dados
del ponto_pvh
del osm_pvh

# Remover point que indica a cidade de pvh
pvh = pvh.drop(pvh[pvh['name'] == 'Porto Velho'].index, axis=0)
pvh.head(5)

/tmp/ipykernel_6340/2645232131.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometry=ponto_pvh.buffer(0.10),


,element_type,osmid,name,geometry,highway,alt_name,oneway,bridge,layer,surface
1,way,28994217,NaN,"LINESTRING (-63.86209 -8.78329, -63.86319 -8.7...",trunk,NaN,yes,NaN,NaN,paved
2,way,28994221,Avenida Rio de Janeiro,"LINESTRING (-63.88116 -8.77129, -63.88024 -8.7...",secondary,NaN,yes,NaN,NaN,NaN
3,way,28994239,Rua Padre Chiquinho,"LINESTRING (-63.90759 -8.74792, -63.90889 -8.7...",residential,NaN,NaN,NaN,NaN,NaN
4,way,28994240,Avenida Farquar,"LINESTRING (-63.90811 -8.76591, -63.90738 -8.7...",tertiary,NaN,yes,NaN,NaN,NaN
5,way,28994243,Avenida Presidente Dutra,"LINESTRING (-63.90911 -8.75026, -63.90910 -8.7...",tertiary,NaN,yes,NaN,NaN,NaN


### Seleção e visualização dos dados
- Será feito um buffer de 0.10 graus (~11km) em volta do ponto que indica o perímetro urbano de Porto Velho.
- Em seguida será feito um filtro dos acidentes que fazem intersecção com o buffer
- Visualizar os dados

In [17]:
import matplotlib.pyplot as plt

#### Visualizando o buffer em relação a cidade

In [ ]:
warnings.simplefilter('ignore', category=DeprecationWarning)

# Plota as ruas mais o buffer
fig, ax = plt.subplots(figsize=(10, 15))
buffer_area_interesse.plot(ax=ax, alpha=.7)
pvh.plot(ax=ax, facecolor='White', edgecolor='black')

#### Fazer o clip e visualizar o dado com as outras camadas

In [ ]:
warnings.simplefilter('ignore', category=DeprecationWarning)

# Recorte os acidentes que estão dentro do buffer
gdf = gdf.clip(buffer_area_interesse.unary_union)

fig, ax = plt.subplots(figsize=(15, 10))

# Zoom no plot
ax.set_ylim([-8.850, -8.675])
ax.set_xlim([-63.975, -63.775])

# Plota a cidade mais buffer
pvh.plot(ax=ax, facecolor='None', edgecolor='black', alpha=.4, linewidth=2, zorder=1)
buffer_area_interesse.plot(ax=ax, facecolor='None', edgecolor='blue', alpha=.3, zorder=2)

# Plota os acidentes
gdf.plot(ax=ax,
         markersize=20,
         column='CONSEQUENCIA',
         marker="^", zorder=3)

plt.show()

### Fazer operações topológicas para gerar ranking de cruzamentos perigosos